In [84]:
import requests
import json

In [36]:
import geopandas
lakesList = './berlin-bb-lakes.geojson'

lakesData = geopandas.read_file(lakesList)
print(lakesData.head())


               id             @id TMC:cid_58:tabcd_1:Class  \
0   relation/4026   relation/4026                     None   
1   relation/4219   relation/4219                     None   
2  relation/27235  relation/27235                     None   
3  relation/48461  relation/48461                     None   
4  relation/55635  relation/55635                     None   

  TMC:cid_58:tabcd_1:LCLversion TMC:cid_58:tabcd_1:LocationCode access  \
0                          None                            None   None   
1                          None                            None   None   
2                          None                            None   None   
3                          None                            None   None   
4                          None                            None   None   

  alt_name bathing  boat canoe  ... sport swimming tidal          type water  \
0     None    None  None  None  ...  None     None  None  multipolygon  lake   
1     None    None  No

In [37]:
lakesData

,id,@id,TMC:cid_58:tabcd_1:Class,TMC:cid_58:tabcd_1:LCLversion,TMC:cid_58:tabcd_1:LocationCode,access,alt_name,bathing,boat,canoe,...,sport,swimming,tidal,type,water,website,wikidata,wikimedia_commons,wikipedia,geometry
0,relation/4026,relation/4026,None,None,None,None,None,None,None,None,...,None,None,None,multipolygon,lake,None,Q63284050,None,None,"POLYGON ((13.20902 52.54121, 13.20901 52.54125..."
1,relation/4219,relation/4219,None,None,None,None,None,None,None,None,...,None,None,None,multipolygon,lake,None,Q1616489,None,de:Hubertussee (Berlin-Grunewald),"POLYGON ((13.28369 52.48544, 13.28369 52.48545..."
2,relation/27235,relation/27235,None,None,None,None,None,None,None,None,...,None,None,None,multipolygon,lake,None,Q1357141,None,de:Liepnitzsee,"POLYGON ((13.49500 52.74325, 13.49510 52.74325..."
3,relation/48461,relation/48461,None,None,None,None,None,None,None,None,...,None,None,None,multipolygon,lake,https://www.seen.de/nymphensee/,Q57416112,None,None,"POLYGON ((13.01985 52.58401, 13.01821 52.58318..."
4,relation/55635,relation/55635,None,None,None,None,None,None,None,None,...,None,None,None,multipolygon,lake,None,None,None,None,"POLYGON ((13.69338 52.58239, 13.69387 52.58235..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,way/991780727,way/991780727,None,None,None,None,None,None,None,None,...,None,None,None,None,lake,None,None,None,None,"POLYGON ((13.80365 52.23826, 13.80359 52.23827..."
238,way/991780728,way/991780728,None,None,None,None,None,None,None,None,...,None,None,None,None,lake,None,None,None,None,"POLYGON ((13.80396 52.23821, 13.80390 52.23821..."
239,way/1018305771,way/1018305771,None,None,None,None,None,None,None,None,...,None,None,None,None,lake,None,None,None,None,"POLYGON ((13.28837 52.69594, 13.28840 52.69590..."
240,way/1034825977,way/1034825977,None,None,None,None,None,None,None,None,...,None,None,None,None,lake,None,Q2354065,None,de:Straussee,"POLYGON ((13.87728 52.58876, 13.87671 52.58859..."


In [42]:
with_entity = 0
without_entity = 0
iterations = 0
for index, lake in lakesData.iterrows():
    iterations = iterations + 1
    # print(lake.wikidata)
    if lake.wikidata != None:
        with_entity = with_entity + 1
    else:
        without_entity = without_entity + 1
    # print(value)


print('iterated over ', iterations, ' objects')
print(with_entity, ' have entities, ', without_entity, ' don`t')


iterated over  242  objects
119  have entities,  123  don`t


In [115]:
queries = []
names = []

for index, lake in lakesData.iterrows():
    if lake.wikidata != None:
       queries.append(lake.wikidata)
       names.append(lake['name'])

len(queries)


119

In [111]:
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

water_portability = {}
# count = 0

def params(query):
    return {
        'action': 'wbgetclaims',
        'entity': query,
        'property': 'P2234',
        'format': 'json'
    }


for index, query in enumerate(queries):
    r = requests.get(API_ENDPOINT, params=params(query))
    response = r.json()
    # count = count + 1
    # print(len(response['claims']))
    if not 'error' in response and 'claims' in response:
        if len(response['claims']) != 0:
        # print(response)
            portability_string = response['claims']['P2234'][0]['mainsnak']['datavalue']['value']['amount']
            portability = int(portability_string[1:].replace('.', ''))
            water_portability[query] = {'portability(m3)': portability, 'name': names[index]}

        else: 
            water_portability[query] = {
                'portability(m3)': 'undefined', 'name': names[index]}
            
    else:
        water_portability[query] = {'portability(m3)': 'undefined', 'name': 'undefined'}



In [112]:
with open('water_portability.json', 'w') as f:
    json.dump(water_portabi lity, f)
